Part 2. RAG (Retrieval-Augmented Generation) 기법


RAG(Retrieval-Augmented Generation) 파이프라인은 기존의 언어 모델에 검색 기능을 추가하여, 주어진 질문이나 문제에 대해 더 정확하고 풍부한 정보를 기반으로 답변을 생성할 수 있게 해줍니다. 이 파이프라인은 크게 데이터 로드, 텍스트 분할, 인덱싱, 검색, 생성의 다섯 단계로 구성됩니다.

Load Data - Text Split - Indexing - Retrieval - Generation

In [ ]:
import os
import langchain

# GOOGLE_API_KEY 설정 (Gemini 라이브러리가 이 이름을 기본으로 찾습니다)
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

langchain.__version__

'1.2.3'

1. 데이터 로드(Load Data)


RAG에 사용할 데이터를 불러오는 단계입니다. 외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드합니다. 예를 들면 공개 데이터셋, 웹 크롤링을 통해 얻은 데이터, 또는 사전에 정리된 자료일 수 있습니다. 가져온 데이터는 검색에 사용될 지식이나 정보를 담고 있어야 합니다.

다음 예제는 langchain_community.document_loaders 모듈에서 WebBaseLoader 클래스를 사용하여 특정 웹페이지(위키피디아 정책과 지침)의 데이터를 가져오는 방법을 보여줍니다. 웹 크롤링을 통해 웹페이지의 텍스트 데이터를 추출하여 Document 객체의 리스트로 변환합니

In [2]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs)) # 문서 갯수
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])


USER_AGENT environment variable not set, consider setting it to identify your requests.


1
13308
총체적으로 어기고 있다면 규범 준수를 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.
문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.
명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만

2. 텍스트 분할(Text Split)

불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 과정입니다. 자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업입니다. 검색 효율성을 높이기 위한 중요한 과정입니다.

다음 코드는 RecursiveCharacterTextSplitter라는 텍스트 분할 도구를 사용하고 있습니다. (이 도구에 대해서는 Text Splitter 챕터에서 상세하게 다룰 예정입니다. ) 간략하게 설명하면 12552 개의 문자로 이루어진 긴 문장을 최대 1000글자 단위로 분할하는 것입니다. 200글자는 각 분할마다 겹치게 하여 문맥이 잘려나가지 않고 유지되게 합니다. 실행 결과를 보면 18개 조각으로 나눠지게 됩니다.

LLM 모델이나 API의 입력 크기에 대한 제한이 있기 때문에, 제한에 걸리지 않도록 적정한 크기로 텍스트의 길이를 줄일 필요가 있습니다. 그리고, 프롬프트가 지나치게 길어질 경우 중요한 정보가 상대적으로 희석되는 문제가 있을 수도 있습니다. 따라서, 적정한 크기로 텍스트를 분할하는 과정이 필요합니다.


In [3]:
# Text Split (Documents -> small chunks: Documents)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10]) # 10번째 스플릿 출력.

# 페이지 콘텐트 속성
# split[10].page_content

# 메타데이터 속성
# splits[10].metadata

18
page_content=''제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다. 
차후 공지가 불충분했다는 이의 제기를 피하려면, 위의 링크를 이용하여 공지하세요. 공지에 비중립적인 단어를 사용하는 등의 선전 행위는 피하세요.' metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'k

결과

18
page_content=''제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다. 
차후 공지가 불충분했다는 이의 제기를 피하려면, 위의 링크를 이용하여 공지하세요. 공지에 비중립적인 단어를 사용하는 등의 선전 행위는 피하세요.' 


metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}

3. 인덱싱(Indexing)
분할된 텍스트를 검색 가능한 형태로 만드는 단계입니다. 인덱싱은 검색 시간을 단축시키고, 검색의 정확도를 높이는 데 중요한 역할을 합니다. LangChain 라이브러리를 사용하여 텍스트를 임베딩으로 변환하고, 이를 저장한 후, 저장된 임베딩을 기반으로 유사성 검색을 수행하는 과정을 보여줍니다. (이 과정에 대해서는 다음 챕터에서 상세하게 다룰 예정입니다. )

간략하게 설명하면 OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터로 변환하고, 이를 Chroma 벡터저장소에 저장합니다. vectorstore.similarity_search 메소드는 주어진 쿼리 문자열("격하 과정에 대해서 설명해주세요.")에 대해 저장된 문서들 중에서 가장 유사한 문서들을 찾아냅니다. 이때 유사성은 임베딩 간의 거리(또는 유사도)로 계산됩니다. 4개의 문서가 반환되는데, 가장 유사도가 높은 첫 번째 문서를 출력하여 확인합니다.

In [4]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
    
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
print(len(docs))
for i, doc in enumerate(docs):
    print(f"\n=== Document {i + 1} ===")
    print(doc.page_content)



Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


4

=== Document 1 ===
재단의 정책이 가장 우선적으로 적용받긴 하지만, 그 외의 일반적인 경우에서 위키백과는 사용자들이 자발적으로 모인 공동체가 운영하는 자치 프로젝트입니다. 정책과 지침은 공동체의 총의와 합의를 반영한 것입니다.
'정책과 지침'이란?
정책 (policy)이란 위키백과의 기본적인 규칙으로 모든 사용자가 기본적으로 따라야 하는 원칙을 의미합니다. 위키백과의 정책 문서는 분류:위키백과 정책 분류에 모여 있습니다. 또한 주요 정책을 요약한 문서로 위키백과:정책 목록 문서도 참고할 수 있습니다.
지침 (guideline)이란 총의에 따라 많은 편집자에게 지지받고 있는 모범 사례를 모아 둔 것입니다. 편집자들은 되도록이라면 상식적인 수준에서 지침을 따르는 것이 좋지만, 때때로 예외적인 상황에선 적용되지 않을 수도 있습니다. 위키백과의 지침 문서는 분류:위키백과 지침 분류에 모여 있습니다. 또한 주요 지침을 요약한 문서로 위키백과:지침 목록 문서도 참고할 수 있습니다.
수필 (essay)이란 광범위한 총의가 형성되지 않은 편집자 혼자, 혹은 편집자 무리의 경험과 의견, 자신만의 해석이나 조언을 모은 것입니다. 정책이나 지침처럼 전체적인 총의를 따르지도, 모든 사용자가 동의하는 내용도 아닐 수 있으며 다른 사람들의 동의 과정 없이도 작성하거나 변경할 수 있습니다. 수필 문서는 분류:위키백과 수필 분류에 모여 있습니다. (자세한 내용은 '위키백과:수필'을 참조하세요)
그 외에 위키백과와 위키프로젝트 이름 공간에 있는 문서의 종류에는 다음이 있습니다.
정책과 지침을 용이하게 적용하기 위한 요청 및 토론 문서.(예를 들어 위키백과:삭제 토론이나 위키백과:관리자 요청이 있습니다)
위키프로젝트 내에서 특정 분야의 문서 작업 효율성 증가를 위해 만든 각종 조언과 자체적인 지침 문서.
위키백과를 사용하는 데 있어 사용 방법에 대한 정보를 얻거나 도움을 받을 수 있는 도움말과 설명문 문서.
위키백과에 대한 기능이나 전체적인 정보를 제공하는 정보문 문서.


4. 검색(Retrieval)
사용자의 질문이나 주어진 컨텍스트에 가장 관련된 정보를 찾아내는 과정입니다. 사용자의 입력을 바탕으로 쿼리를 생성하고, 인덱싱된 데이터에서 가장 관련성 높은 정보를 검색합니다. LangChain의 retriever 메소드를 사용합니다.

5. 생성(Generation)
검색된 정보를 바탕으로 사용자의 질문에 답변을 생성하는 최종 단계입니다. LLM 모델에 검색 결과와 함께 사용자의 입력을 전달합니다. 모델은 사전 학습된 지식과 검색 결과를 결합하여 주어진 질문에 가장 적절한 답변을 생성합니다.

검색과 생성 단계를 수행하는 다음 코드를 살펴보겠습니다. vectorstore.as_retriever() 메소드는 Chroma 벡터 스토어를 검색기로 사용하여 사용자의 질문과 관련된 문서를 검색합니다. format_docs 함수는 검색된 문서들을 하나의 문자열로 반환합니다. RAG 체인을 구성하고, 주어진 질문에 대한 답변을 생성합니다.

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI # 변경
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM, 일관성을 위해 temperature를 0으로 설정
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite', temperature=0) 

# Rretriever   
retriever = vectorstore.as_retriever()

# Combine Documents 문서를 합치기
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


'제공된 맥락에서는 "격하 과정"에 대한 설명이 없습니다.'

결과

'제공된 문맥에는 "격하 과정"에 대한 구체적인 설명이 포함되어 있지 않습니다. 따라서 격하 과정에 대한 정보를 제공할 수 없습니다. 추가적인 정보나 다른 질문이 있다면 말씀해 주세요.'